# RetinaNet (Focal Loss for Dense Object Detection)  
- 논문 리뷰

## Abstract  
- 높은 정확도를 가진 detector는 two-stage method model이다  
- 반면 one-stage detector model은 더 빠르지만 정확도가 떨어진다  
- 우리는 이것이 foreground - background의 심각한 class imbalance가 원인임을 찾았다  
- 잘 분류된 class에 대해서는 down-weight를 적용해 Cross Entropy Loss를 수정함으로써 class imbalance를 해결한다  
- 그래서 hard example에 대해서 집중적으로 학습하는 Focal Loss를 제안한다  
- 우리의 loss function을 평가하기 위해 간단한 dense detector를 디자인했으며 이를 RetinaNet이라 한다  
- focal loss로 학습을 시키면 RetinaNet은 one-stage detector만큼 빠르며 two-stage detector만큼 정확한 성능을 보여준다  

## Introduction  
- 현재의 SOTA object detector는 proposal mechanism의 two-stage method를 따른다  
- first stage는 candidate object location을 생성하고 second stage는 각 candidate location을 분류한다  
- 최근 YOLO나 SSD와 같은 one-stage detector에 대한 연구는 흥미로운 결과를 보여주고 two-stage detector에 비해 굉장히 빠른 속도를 보여준다  
- one-stage detector만큼 빠르고 two-stage detector만큼 정확한 detector를 제안한다  
- 이를 위해 우리는 훈련하는 동안 방해가 되는 요소인 class imbalance 문제를 해결하고자 한다  
- R-CNN 계열 모델들은 Region Proposal이 적은 수의 object location을 생성해 대부분의 background sample을 제거한다  
- second stage에서는 foreground와 background의 비율이 1:3을 유지해 class balance를 관리할 수 있다  
- 반대로 one-stage detector는 더 많은 candidate object location set을 처리한다  
- 이미지 내에서 scale와 aspect ratio를 이용해 생성되는 candidate의 수가 너무 많이 학습에 비효율적이다  
- 그래서 이 논문에서 우리는 이 문제점을 해결할 수 있는 Focal Loss를 제안한다  
- 직관적으로 Focal Loss의 scaling factor는 easy example에 대한 weight를 낮춘다  
- 따라서 hard example에 대해 집중적으로 학습을 할 수 있다  
- Focal Loss의 효과를 보여주기 위해 우리는 RetinaNet이라는 one-stage detector를 구축했다  
- RetinaNet은 효율적이고 정확하며 ResNet101-FPN을 backbone으로 사용한 경우, COCO test set에서 5 fps를 끌어올렸다  


## Focal Loss  
- Focal Loss는 one-stage object detection scenario에서 발생하는 극단적인 class imbalance 문제를 해결하기 위해 디자인되었다  

$$CE\left(p,y\right)=\left\{\begin{array}{cc} -\log\left(p\right)\\-log\left(1-p\right) \end{array}\right.$$  

$$p_t=\left\{\begin{array}{cc} p\\1-p\end{array}\right.$$    

- CE Loss는 figure 1에서 파란색 그래프에 해당한다  
- 여기서 주목할 점은 $p_t$가 0.5보다 클 때 loss 값이 꽤 크다는 점이다  
- 이러한 loss가 쌓이면 결국 모델이 잘 학습되지 않을 수 있다  

### Balanced Cross Entropy  
- class imbalance를 해결하는 방법 중 하나로 weighted factor $\alpha\in\left[0,1\right]$을 도입했다  

$$CE\left(p_t\right)=-\alpha_t\log\left(p_t\right)$$  

- 이 아이디어를 이용해 Focal Loss를 만들었다  

### Focal Loss Definition  
- loss의 대부분은 easy negative로 구성되며 gradient 또한 이들이 지배한다  
- weighted CE Loss는 $\alpha$로 positive와 negative sample의 weight를 조절할 수 있지만 easy와 hard sample을 구별하지는 못한다  
- 그래서 우리는 easy example에 대해서는 down-weight를 하고 hard example에 대해서는 집중적으로 학습하도록 수정된 loss function을 제안한다  

$$FL\left(p_t\right)=-\left(1-p_t\right)^{\gamma}\log\left(p_t\right)$$  

- 우리는 Focal Loss의 두 가지 특성에 주목한다  
    - 1. 확률 $p_t$ 값이 작을수록 modulating factor는 1에 가까워지며 loss 값이 커진다  
    - 반대로 $p_t$ 값이 클수록 modulating factor는 0에 가까워지며 well-classified example의 loss 값이 작아진다  
    - 2. $\gamma$ 값을 조절해 easy example에 대한 down-weighted 정도를 조절할 수 있다  
    - 논문에서는 $\gamma=2$일 때 가장 성능이 좋았다고 한다  
    - $\gamma$ 값이 커질수록 modulating factor의 영향력이 커진다  
    
- 실제로는 $\alpha$-balanced focal loss를 사용한다  

$$FL\left(p_t\right)=-\alpha_t\left(1-p_t\right)^{\gamma}\log\left(p_t\right)$$  

- $\alpha$를 적용한 것이 안한 것보다 더 성능이 좋았다  

### Class Imbalance and Model Initialization  
- Binary Classification에서는 기본적으로 output의 확률 값을 0.5, 0.5로 놓는다  
- 이렇게 설정하면 class imbalance의 존재로 인해 빈도가 많은 class의 loss가 전체 loss를 지배하게 된다  
- 이를 해결하기 위해 rare class에 대한 prior value를 도입한다  
- 이 prior를 $\pi$라 정의하며 prior 값은 예를 들어 0.01로 설정할 수 있다  
- 이런 prior value $\pi$ 값으로 인해 모델이 더 안정적으로 학습할 수 있다는 것을 발견했다  

### Class Imbalance and Two-stage Detectors  
- Two-stage detector는 class imbalance를 다음과 같이 해결한다  
    - 1. two-stage cascade  
    - 2. bias mini batch sampling  
    
- 먼저 cascade stage는 possible object location을 무한대에서 1~2000개로 줄여준다  
- 두 번째로 bias sampling은 positive와 negative sample의 비율을 1:3으로 조절해준다  

## RetinaNet Detector  
- RetinaNet은 backbone network와 two task subnet으로 이루어져 있다  
- 첫 번째 subnet은 backbone의 output에서 convolutional object classification을 진행한다  
- 두 번째 subnet은 convolutional bounding box regression을 진행한다  

**Feature Pyramid Network Backbone**  
- FPN은 하나의 이미지에 대해 multi-scale feature pyramid를 생성한다  
- 각 pyramid는 다양한 scale의 object를 detect한다  
- FPN은 ResNet의 상단에 적용시킨다  
- FPN 없이 ResNet만 사용한 것보다 FPN을 함께 사용한 것이 더 성능이 좋았다  

**Anchors**  
- anchor box의 aspect ratio는 $\left\{1:2, 1:1, 2:1\right\}$을 사용한다  
- scale로는 $\left\{2^0, 2^{1/3}, 2^{2/3}\right\}$을 사용한다  
- 그래서 총 9개의 anchor를 사용했고 scale의 범위는 32~813 pixel을 사용했다  
- IoU가 0.4 미만인 경우는 background로 정의하고 $\left[0.4, 0.5\right)$인 경우는 무시한다  

**Classification Subnet**  
- classification subnet은 각 position에 object가 존재할 확률을 예측한다  
- 이 subnet은 각 FPN level에서 붙여진 작은 FCN이다  
- pyramid level에서 channel $C$의 feature map을 얻으면 subnet은 4개의 $3\times 3$ conv layer와 ReLU를 적용한다  
- 이때 channel은 KA이다  

**Box Regression Subnet**  
- box regression subnet은 classification subnet과 병렬로 수행되며 각 pyramid level에 또 다른 작은 FCN을 붙여준다  
- box regression subnet은 box의 좌표 4개의 값을 예측한다  
- classification subnet과 box regression subnet은 같은 구조를 가지지만 별개의 parameter를 사용한다  


### Inference and Training  
**Inference**  
- RetinaNet은 ResNet-FPN backbone과 classification subnet, box regression subnet으로 구성되어 있다  
- 속도를 향상시키기 위해 score가 높은 기준으로 1000개의 box만 사용한다  
- top prediction에 threshold 0.5와 NMS를 적용해 최종 결과를 출력한다  

**Focal Loss**  
- 우리는 classification subnet에 focal loss를 사용한다  
- 또한 $\gamma=2$일 때가 가장 좋은 결과가 나았으며 $\gamma\in\left[0.5,0.5\right]$일 때 RetinaNet이 robust한 경향을 보인다  
- OHEM, SSD는 일부 anchor (ex. 256)에만 적용이 되는 반면 우리의 focal loss는 샘플링된 모든 anchor에 적용이 된다  
- rare class의 weight인 $\alpha$는 $\gamma$와 서로 상호작용하며 적절한 값을 골라야 한다  
- $\alpha=0.25$, $\gamma=2$일 때가 가장 성능이 좋았다  

**Initialization**  
- 우리는 ResNet50-FPN과 ResNet101-FPN을 실험했다  
- 이때 ResNet은 ImageNet1K로 사전 훈련된 모델을 사용한다  
- RetinaNet subnet에서 마지막 layer를 제외한 모든 conv layer는 bias는 0, gaussian weight fill은 0.01로 초기화한다  
- 마지막 layer에는 $b=-\log\left(\frac{1-\pi}{\pi}\right)$로 초기화한다  
- 이때 $\pi = 0.01$이다  

**Optimization**  
- optimizer : SGD momentum 0.9  
- batch size : 16 (2 images per GPU, 8 GPUs)  
- iterations : 90k  
- lr : 0.01 (which divided by 10 at 60k and again at 80k iterations)  
- weight decay : 0.0001  
- training loss는 focal loss와 box regression loss를 더한 값으로 계산  
- 이때 box regression loss는 standard smooth L1 loss  

## Experiments  
### Training Dense Detection  
- 모든 실험에서 ResNet50과 101 마지막에 FPN을 추가한 backbone을 사용했다  

**Network Initialization**  
- 먼저 Cross Entropy Loss로 학습을 시켜봤지만 성능이 매우 안좋았다  
- RetinaNet은 COCO에 대해 30.2의 AP를 산출한다  
- $\pi$값에 따라 성능이 크게 좌지우지되지 않아 $\pi=0.01$을 이용한다  

**Balanced Cross Entropy**  
- table 1(a)에 balanced CE Loss를 사용한 결과가 있다  

**Focal Loss**  
- $\gamma$가 증가할수록 focal loss의 성능이 CE Loss의 성능을 뛰어넘는다  
- $\gamma=2$일 때 $\alpha$-balanced CE Loss인 경우보다 2.9 point의 AP를 더 얻었다  
- 전체적으로 $\gamma$가 클수록 $\alpha\in\left[0.25, 0.75\riht]$의 범위가 좋은 결과를 보여준다  
- 우리는 모든 실험에서 $\gamma=2$, $\alpha=0.25$를 사용했다  

**Analysis of the Focal Loss**  
- figure 4에 positive와 negative sample에 대한 누적 분포 함수를 보여주고 있다  
- positive sample에 대한 CDF를 보면 $\gamma$ 값이 달라고 비슷한 그래프를 보여주고 있다  
- 그러나 negative sample에 대한 CDF를 보면 전혀 다른 형태를 가진다  
- $\gamma=0$일 때 positive와 negative의 그래프가 비슷하다  
- 그러나 $\gamma$가 증가할수록 hard negative에 weight가 더 많이 집중하는 것을 알 수 있다  
- 이렇듯이 Focal Loss는 easy negative는 무시하면서 hard negative에 대해 더 많이 학습할 수 있다  

**Online Hard Example Mining (OHEM)**  
- OHEM Loss는 focal loss와 비슷하게 잘못 분류된 example에 대해 학습을 집중적으로 한다  
- 그러나 easy example에 대한 loss는 완전히 버린다는 점이 차이점이다  
- OHEM Loss로 Focal Loss와 똑같은 환경에서 성능을 비교해본 결과 OHEM이 3.2 AP가 떨어진다  
- 이는 Focal Loss가 더 효과적이라는 것을 보여준다  

### Model Architecture Design  
**Anchor Density**
- one-stage detector에서 가장 중요한 것 중 하나는 anchor box가 이미지를 얼마나 촘촘하게 덮느냐이다  
- 다양한 크기의 object를 찾아내기 위해선 다양한 scale의 anchor box를 사용하는 것이다  
- 이 논문에서는 square anchor만을 사용해도 30.3 AP의 좋은 성능을 보여준다  
- 또한 3 scale과 3 aspect ratio를 사용해도 AP를 약 4 포인트 개선할 수 있다  
- 마지막으로 anchor의 갯수가 6~9개를 초과해도 더 성능이 좋아지진 않았다  

**Speed versus Accuracy**  
- backbone이 클수록 높은 정확도를 산출하지만 속도가 더 느리다  
- figure 2에서 속도와 정확도의 trade-off를 보여주고 있다  
- ResNet101 FPN을 사용한 RetinaNet은 Faster R-CNN과 비슷한 정확도를 보인다  
- RetinaNet의 scale을 더 키워도 two-stage detector의 성능을 능가하며 여전히 빠른 속도를 가진다  

### Comparision to State of the Art  
- one-stage detector과 비교하면 5.9 AP point 더 좋은 성능을 얻는다  
- two-stage detector과 비교해도 2.3 AP point 더 좋은 성능을 얻는다  

## Conclusion  
- 우리는 one-stage detector가 two-stage detector의 정확도를 뛰어넘지 못하는 이유가 class imbalance임을 확인했다  
- 이를 위해 우리는 focal loss를 제안했으며 이는 hard negative sample을 집중적으로 학습한다  
- 우리가 설계한 RetinaNet으로 성능을 측정한 결과, SOTA의 speed와 accuracy를 달성했다  